In [ ]:
# this file downloads xml files from pubmed ftp
# fisrt, get a list of target files on the pubmed server
# then, for each file, do the following:
#  - access ftp and download the xml file
#  - do the parsing and output the dataframe
#  - delete the xml file (to save space on local disk)
# note: we access ftp once per file since the processing of one file can take some time, and 
# ftp might lose connection during this period of inactivation

In [1]:
import ftplib
import os
# uncomment this line on rcc !pip install --user git+http://github.com/titipata/pubmed_parser.git
import pubmed_parser as pp
import pandas as pd
import pickle

import nltk
from nltk.corpus import stopwords
import numpy as np

In [65]:
ftp = ftplib.FTP("ftp.ncbi.nlm.nih.gov")
ftp.login("anonymous", "mlyu@uchicago.edu")

'230 Anonymous access granted, restrictions apply'

In [66]:
ftp.cwd("/pubmed/updatefiles")
files = ftp.nlst() # get list of filenames
ftp.quit()

'221 Goodbye.'

In [67]:
files = [f for f in files if f.endswith("xml.gz")]
files # len 81

['pubmed18n0929.xml.gz',
 'pubmed18n0930.xml.gz',
 'pubmed18n0931.xml.gz',
 'pubmed18n0932.xml.gz',
 'pubmed18n0933.xml.gz',
 'pubmed18n0934.xml.gz',
 'pubmed18n0935.xml.gz',
 'pubmed18n0936.xml.gz',
 'pubmed18n0937.xml.gz',
 'pubmed18n0938.xml.gz',
 'pubmed18n0939.xml.gz',
 'pubmed18n0940.xml.gz',
 'pubmed18n0941.xml.gz',
 'pubmed18n0942.xml.gz',
 'pubmed18n0943.xml.gz',
 'pubmed18n0944.xml.gz',
 'pubmed18n0945.xml.gz',
 'pubmed18n0946.xml.gz',
 'pubmed18n0947.xml.gz',
 'pubmed18n0948.xml.gz',
 'pubmed18n0949.xml.gz',
 'pubmed18n0950.xml.gz',
 'pubmed18n0951.xml.gz',
 'pubmed18n0952.xml.gz',
 'pubmed18n0953.xml.gz',
 'pubmed18n0954.xml.gz',
 'pubmed18n0955.xml.gz',
 'pubmed18n0956.xml.gz',
 'pubmed18n0957.xml.gz',
 'pubmed18n0958.xml.gz',
 'pubmed18n0959.xml.gz',
 'pubmed18n0960.xml.gz',
 'pubmed18n0961.xml.gz',
 'pubmed18n0962.xml.gz',
 'pubmed18n0963.xml.gz',
 'pubmed18n0964.xml.gz',
 'pubmed18n0965.xml.gz',
 'pubmed18n0966.xml.gz',
 'pubmed18n0967.xml.gz',
 'pubmed18n0968.xml.gz',


In [68]:
files = files[71:80] #### debug ####
#files = ["pubmed18n0949.xml.gz"]

In [69]:
files

['pubmed18n1000.xml.gz',
 'pubmed18n1001.xml.gz',
 'pubmed18n1002.xml.gz',
 'pubmed18n1003.xml.gz',
 'pubmed18n1004.xml.gz',
 'pubmed18n1005.xml.gz',
 'pubmed18n1006.xml.gz',
 'pubmed18n1007.xml.gz',
 'pubmed18n1008.xml.gz']

In [70]:
# exclude stopwords (need to be editted if wanted stemmer)
def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None):

    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)

    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)
     # return a list with the stopwords removed
    return list(workingIter)

def psycfilter(normalLst):
    result = (w for w in normalLst if w in new_dict)
    return list(result)

In [71]:
for filename in files:
    # access ftp
    ftp = ftplib.FTP("ftp.ncbi.nlm.nih.gov")
    ftp.login("anonymous", "mlyu@uchicago.edu")
    ftp.cwd("/pubmed/updatefiles")
    # download xml file
    with open(filename, "wb") as f:
        ftp.retrbinary("RETR " + filename, f.write)
    ftp.quit()
    # do parsing
    pubmed_parsed = pp.parse_medline_xml("./" + filename)
    pubmed_df = pd.DataFrame(pubmed_parsed)
    
    # filter out relevant rows
    new_dict = ['psycholog', 'behavior','neurosci']

    stop_words_nltk = stopwords.words('english')
    porter = nltk.stem.porter.PorterStemmer()
    snowball = nltk.stem.snowball.SnowballStemmer('english')

    # tokenize abstract
    not_delete = ~pubmed_df['delete'] 
    pubmed_df.loc[not_delete,'tokenized_abstract'] = pubmed_df.loc[not_delete,'abstract'].apply(nltk.word_tokenize) 

    # fliter out deleted=TRUE pubmed
    pubmedDF = pubmed_df[~pubmed_df['abstract'].isnull()].copy()

    # tokenize title
    pubmedDF['tokenized_title'] = pubmedDF['title'].apply(lambda x: nltk.word_tokenize(x))



    pubmedDF['normalized_abstract'] = pubmedDF['tokenized_abstract'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))
    pubmedDF['normalized_title'] = pubmedDF['tokenized_title'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))

    pubmed_new = pubmedDF[pubmedDF.abstract != ""] # exclude those without abstract

    pubmed_new['psyc_check'] = pubmed_new['normalized_abstract'].apply(lambda x: psycfilter(x))
    pubmed_new['psyc_filter'] = pubmed_new['psyc_check'].apply(lambda x: True if len(x) > 0 else False)
    pubmed_filtered = pubmed_new[pubmed_new['psyc_filter'] == True]
    
    # save df to local
    pubmed_filtered.to_csv("./intermediate_data/" + filename[:-7] + ".csv")
    #with open("./intermediate_data/" + filename[:-7] + ".pkl", "wb") as f:
        #pickle.dump(pubmed_filtered, f) # to reload: df = pd.read_pickle(filename)
    # delete xml file
    os.remove(filename)

C:\Users\lyumu\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lyumu\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
pubmed_df = pd.read_pickle("./intermediate_data/pubmed18n0929.pkl")
pubmed_df

,abstract,affiliation,author,country,delete,issn_linking,journal,keywords,medline_ta,mesh_terms,nlm_unique_id,other_id,pmc,pmid,pubdate,title
0,"A simple, highly sensitive and reproducible me...",,H Tamaoki; S Minato; S Takei; K Fujisawa,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D000328:Adult; D001660:Biliary Tract Diseases;...,1302422,,,165,1975,A clinical method for the determination of ser...
1,The simplification of the measurement of circu...,,E Rolleri; M Zannino; S Orlandini; R Malvano,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D000324:Adrenocorticotropic Hormone; D002151:C...,1302422,,,2396,1976,Direct radioimmunoassay of plasma cortisol.
2,Simplification of radioimmunoassay procedures ...,,R Malvano; S Orlandini; P Cozzani; P Duranti; ...,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D000450:Aldosterone; D000818:Animals; D003429:...,1302422,,,2397,1976,Methodological simplifications in radioimmunoa...
3,Peptidases activities were compared in human l...,,A Scharfman; P Roussel; G Biserte; C Aerts; AB...,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D000818:Animals; D006168:Guinea Pigs; D006801:...,1302422,,,2398,1976,[Studies on the activities of peptidases from ...
4,,,S Standing; CP Price,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D005069:Evaluation Studies as Topic; D006242:H...,1302422,,,2399,1976,A kinetic method for the determination of hapt...
5,,,LP Hackett; LJ Dusci,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,"D002849:Chromatography, Gas; D006801:Humans; D...",1302422,,,2400,1976,Gas-liquid chromatographic determination of su...
6,Fractionations of serum gamma-glutamyltranspep...,,CP Degenaar; C Thijssen; G Van der Wal; GT Ber...,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,"D008052:Lipid Metabolism, Inborn Errors; D0080...",1302422,,,3301,1976,Electrophoresis of gamma-glutamyltranspeptidas...
7,Acid-thermostable ribonucleases were isolated ...,,A Bardoń; H Slerakowska; D Shugar,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D002648:Child; D003300:Copper; D003550:Cystic ...,1302422,,,4243,1976,Purification and properties of human acid-ther...
8,,,A Saifer; J Amoroso; G Perle,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D005838:Genotype; D006596:Hexosaminidases; D00...,1302422,,,4244,1976,Urinary hexosaminidase analysis for the identi...
9,The physico-chemical and immunological propert...,,JF Koster; RG Slee; JM Van der Klei-Van Moorse...,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,,Clin Chim Acta,D000937:Antigen-Antibody Reactions; D004355:Dr...,1302422,,,4245,1976,Physico-chemical and immunological properties ...
